In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np

# MODIFY: Define the classes and vars for prediction
classes = ["Art_Nouveau_Modern", "Baroque", "Expressionism", "Impressionism", "Post_Impressionism", 
           "Realism", "Renaissance", "Romanticism", "Symbolism", 
           "Ukiyo_e"]

input_size = 2048
num_classes = len(classes)
classifier_path = 'model_params.pt'
device = 'cpu'
topx = 2

In [19]:
# MODIFY: Neural Net definition
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.fc1=nn.Linear(input_size, input_size*8)
        self.fc2=nn.Linear(input_size*8, input_size*2)
        self.fc3=nn.Linear(input_size*2, input_size//8)
        self.outputLayer=nn.Linear(input_size//8, num_classes)
        self.relu=nn.ReLU()

    def forward(self, x):
        x=x.view(-1, self.input_size)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x=self.relu(x)
        x=self.fc3(x)
        x=self.relu(x)
        x=self.outputLayer(x)
        return x

In [20]:
# Load models globally at app startup
resnet = models.resnet152(weights='ResNet152_Weights.DEFAULT')
feature_extractor = nn.Sequential(*list(resnet.children())[:-1]).to(device).eval()

# Load classifier model once at the start
classifier = NeuralNet().to(device)
print(classifier)
classifier.load_state_dict(torch.load(classifier_path, map_location=torch.device(device), weights_only=True))
classifier.eval()  # Set the classifier model to evaluation mode

NeuralNet(
  (fc1): Linear(in_features=2048, out_features=16384, bias=True)
  (fc2): Linear(in_features=16384, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=256, bias=True)
  (outputLayer): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
)


NeuralNet(
  (fc1): Linear(in_features=2048, out_features=16384, bias=True)
  (fc2): Linear(in_features=16384, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=256, bias=True)
  (outputLayer): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
)

In [21]:
# MODIFY: Change this is extract features method is changed
def extract_features(image_file):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    img = Image.open(image_file).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = feature_extractor(img_tensor)
        features = features.view(features.size(0), -1)
    return features.cpu().numpy()

In [22]:
def classify_image(features):
    features_tensor = torch.tensor(features).to(device)
    with torch.no_grad():
        output = classifier(features_tensor)
        probabilities = F.softmax(output, dim=1)
        top_classes, top_index = torch.topk(probabilities, topx)
    return top_index.cpu().numpy()[0], top_classes.cpu().numpy()[0]

In [26]:
def runMain(image_file):
    features = extract_features(image_file)
    # Get the top x predicted classes
    top_indices, top_classes = classify_image(features)

    # Dynamically create a dictionary for top_x results
    predictions = {}
    for i in range(topx):
        predictions[f'top_{i+1}'] = {
            'class': classes[top_indices[i]],
            'score': float(top_classes[i])
        }
    return predictions

file = 'image.jpg'
prediction = runMain(image_file=file)
# Pretty print without using json
for key, value in prediction.items():
    print(f"{key}:")
    for sub_key, sub_value in value.items():
        print(f"    {sub_key}: {sub_value}")

top_1:
    class: Expressionism
    score: 0.9999767541885376
top_2:
    class: Art_Nouveau_Modern
    score: 2.3148590116761625e-05
